# Processing statistics files

In this example we show how one can use pysemtools to average fields in space.

#### Import general modules

In [9]:
# Import required modules
from mpi4py import MPI #equivalent to the use of MPI_init() in C
import matplotlib.pyplot as plt
import numpy as np

# Get mpi info
comm = MPI.COMM_WORLD

# Hide the log for the notebook. Not recommended when running in clusters as it is better you see what happens
import os
os.environ["PYSEMTOOLS_HIDE_LOG"] = 'false'


### Clone the data

Get some data from a public repository

In [10]:
# Clone the repository with the data
os.system("git clone https://github.com/adperezm/sem_data.git ../data/sem_data")

fatal: destination path '../data/sem_data' already exists and is not an empty directory.


32768

## Create a file name sequence

This file name sequence will be used to read and average the data.

In [11]:
n_files = 1
folder_path = "../data/"
output_path = "../data/"
file_sequence =[folder_path + f"tc_channel0.f{str(i+1).zfill(5)}" for i in range(n_files)]

### Average the file sequence in the homogeneous direction

Simply call the function that performs the average

In [12]:
from pysemtools.postprocessing.statistics.fld_stats import generate_augmented_field
from pysemtools.datatypes.msh import Mesh
from pysemtools.datatypes.field import FieldRegistry
from pysemtools.datatypes.coef import Coef
from pysemtools.io.ppymech.neksuite import pynekread, pynekwrite

# Read the data
bckend = "torch"
msh = Mesh(comm=comm, bckend=bckend)
pynekread(comm=comm, filename = file_sequence[0], msh = msh, data_dtype=np.float32) # Always define the data type in numpy.

# Generate the coefficients
coef = Coef(msh=msh, comm=comm, bckend=bckend)

for i, fname in enumerate(file_sequence):

    if comm.Get_rank() == 0:
        print("===============================================")
    
    fld = FieldRegistry(comm=comm, bckend=bckend)
    pynekread(comm=comm, filename = fname, fld = fld, data_dtype=np.float32) # Always define the data type in numpy.

    # call the routine
    augmented_fld = generate_augmented_field(comm=comm, msh=msh, fld=fld, coef=coef, dtype=msh.x.dtype)

    # Write the data
    pynekwrite(comm=comm, filename = output_path + f"augmented_field0.f{str(i+1).zfill(5)}", msh = msh.to(comm = comm, bckend="numpy"), fld = augmented_fld.to(comm = comm, bckend="numpy"), wdsz=4)

2025-03-02 18:13:42,540 - Mesh - INFO - Initializing empty Mesh object.
2025-03-02 18:13:42,542 - pynekread - INFO - Reading file: ../data/tc_channel0.f00001
2025-03-02 18:13:42,546 - Mesh - INFO - Initializing Mesh object from x,y,z ndarrays.
2025-03-02 18:13:42,547 - Mesh - INFO - Initializing common attributes.
2025-03-02 18:13:42,548 - Mesh - INFO - Getting vertices
2025-03-02 18:13:42,548 - Mesh - INFO - Getting edge centers
2025-03-02 18:13:42,555 - Mesh - INFO - Getting facet centers
2025-03-02 18:13:42,561 - Mesh - INFO - Mesh object initialized.
2025-03-02 18:13:42,563 - Mesh - INFO - Mesh data is of type: torch.float32
2025-03-02 18:13:42,563 - Mesh - INFO - Elapsed time: 0.01679431399998066s
2025-03-02 18:13:42,564 - pynekread - INFO - File read
2025-03-02 18:13:42,569 - pynekread - INFO - Elapsed time: 0.02723331499998949s
2025-03-02 18:13:42,570 - Coef - INFO - Initializing Coef object
2025-03-02 18:13:42,571 - Coef - INFO - Getting derivative matrices
2025-03-02 18:13:42,

That's it! check your file in, e.g., Visit or paraview and check how it turned out!